In [4]:
print("oke")

oke


In [5]:
%pwd

'c:\\UNIVERSITY\\SEMESTER 4\\Artificial Intelligence\\Project\\Chatbot\\End-To-End-Medical-Chatbot\\research'

In [6]:
import os
os.chdir("../")

In [7]:
%pwd

'c:\\UNIVERSITY\\SEMESTER 4\\Artificial Intelligence\\Project\\Chatbot\\End-To-End-Medical-Chatbot'

In [8]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
# Extract data from the PDF files
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob = "*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents


In [10]:
# import pandas as pd

# # Extract data from CSV files
# def load_csv_file(data):
#     # Load all CSV files from the specified directory
#     csv_files = DirectoryLoader(data, 
#                                 glob="*.csv", 
#                                 loader_cls=pd.read_csv)
    
#     # Read and concatenate data from all CSV files
#     documents = []
#     for file in csv_files.load():
#         df = pd.read_csv(file)  # Load each CSV file into a DataFrame
#         documents.append(df)
    
#     return pd.concat(documents, ignore_index=True)  # Concatenate all DataFrames into one


In [11]:
extracted_data = load_pdf_file(data='./Data')

In [12]:
# extracted_data

In [13]:
# Split the Data in Chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [14]:
text_chunks = text_split(extracted_data)
print("length of text_chunks: ", len(text_chunks))

length of text_chunks:  5990


In [15]:
# text_chunks

In [16]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [17]:
# Download the embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [18]:
embeddings = download_hugging_face_embeddings()

c:\Users\Lenovo\anaconda3\envs\medical\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [19]:
query_result = embeddings.embed_query("hello")
print("length of query_result: ", len(query_result))

length of query_result:  384


In [20]:
# query_result

In [21]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

In [27]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-chatbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'eb17b2d20b01c7f40d428fd7ba7f1dec', 'Date': 'Thu, 07 Nov 2024 15:36:44 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [28]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [29]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)

In [30]:
# Load existing index
from langchain_pinecone import PineconeVectorStore

# Embed each chunk and upsert the embeddings into Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
) 

In [31]:
docsearch

In [32]:
retreiver = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [33]:
retreived_docs = retreiver.invoke("What is President University?")

In [34]:
retreived_docs

[Document(id='5cd6326d-d123-44d9-8dfc-1021001ac8c8', metadata={'page': 17.0, 'source': 'Data\\COVID-19_datasets_A_brief_overview.pdf'}, page_content='University, India respectively. She is currently a Scholarly Teaching Fellow in Global Pro-\nfessional School, Federation University Australia. Her current research interests include\nMachine Learning, Artificial Intelligence, String Matching, and Data/Text Mining.\nMehmood Chadhar received his PhD in Information Systems from the University of\nNew South Wales, Sydney, Australia. He is currently a Lecturer teaching business analyt-'),
 Document(id='4722a773-d283-4cf9-bb28-d1da4993bc25', metadata={'page': 17.0, 'source': 'Data\\COVID-19_datasets_A_brief_overview.pdf'}, page_content='University, India respectively. She is currently a Scholarly Teaching Fellow in Global Pro-\nfessional School, Federation University Australia. Her current research interests include\nMachine Learning, Artificial Intelligence, String Matching, and Data/Text Min

In [35]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0.4,
    max_retries=2,
    max_tokens=512,
    
)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an medical AI assistant for answering question about disease."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, just say"
    "you don't know. Use three sentence maximum and keep the"
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [36]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retreiver, question_answer_chain)

In [47]:
response = rag_chain.invoke({"input": "Could you tell me why I feel hurt in my ear?"})
print(response["answer"])

Based on the provided context, feeling hurt in the ear is not explicitly mentioned. However, acoustic neuromas can cause hearing loss and affect the vestibular nerve, leading to symptoms like vertigo and dysequilibrium. It is recommended to consult a healthcare professional for a proper evaluation of ear pain.
